In [21]:
import pandas as pd
import numpy as np

# ===============================
# 1️⃣ LOAD FILE
# ===============================
file_path = (r"C:\Users\hoora\OneDrive\Desktop\Project 3\Global_Automotive_Analytics_1000_Records.xlsx")
sheets = pd.read_excel(file_path, sheet_name=None)

df_dealer = sheets["Dealership_Master"]
df_car = sheets["Car_Master"]
df_sales = sheets["Sales_Transactions"]
df_cost = sheets["Cost_Profit"]
df_market = sheets["Market_Factors"]

print("✅ File Loaded Successfully")

✅ File Loaded Successfully


In [23]:
for name, df in sheets.items():
    print(f"\nSheet Name: {name}")
    print("Shape:", df.shape)
    print(df.head())


Sheet Name: Dealership_Master
Shape: (6, 7)
  Dealer_ID             Dealer_Name          City   Country Market_Region  \
0      D001   Toyota Karachi Motors       Karachi  Pakistan      Pakistan   
1      D002    Honda Karachi Center       Karachi  Pakistan      Pakistan   
2      D003       BMW Dubai Premium         Dubai       UAE          MENA   
3      D004      Kia Abu Dhabi Auto     Abu Dhabi       UAE          MENA   
4      D005  Tesla Kuala Lumpur Hub  Kuala Lumpur  Malaysia           SEA   

  Dealer_Type  Opening_Year  
0   Corporate          2012  
1   Franchise          2015  
2   Corporate          2018  
3   Franchise          2016  
4   Corporate          2020  

Sheet Name: Car_Master
Shape: (6, 8)
  Car_ID   Brand     Model Segment Fuel_Type Transmission  Engine_CC  \
0   C001  Toyota   Corolla   Sedan    Petrol    Automatic       1800   
1   C002   Honda     Civic   Sedan    Petrol    Automatic       1500   
2   C003     BMW        X5     SUV    Petrol    Automatic 

In [25]:
for df in [df_dealer, df_car, df_sales, df_cost, df_market]:
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].str.strip()

In [28]:
for name, df in {
    "Dealer": df_dealer,
    "Car": df_car,
    "Sales": df_sales,
    "Cost": df_cost,
    "Market": df_market
}.items():
    
    print(f"\nChecking sheet: {name}")
    
    for col in df.select_dtypes(include=['object', 'string']).columns:
        space_issue = df[col].str.startswith(" ").sum() + df[col].str.endswith(" ").sum()
        
        if space_issue > 0:
            print(f"❌ Column '{col}' still has {space_issue} space issues")
        else:
            print(f"✅ Column '{col}' is clean")


Checking sheet: Dealer
✅ Column 'Dealer_ID' is clean
✅ Column 'Dealer_Name' is clean
✅ Column 'City' is clean
✅ Column 'Country' is clean
✅ Column 'Market_Region' is clean
✅ Column 'Dealer_Type' is clean

Checking sheet: Car
✅ Column 'Car_ID' is clean
✅ Column 'Brand' is clean
✅ Column 'Model' is clean
✅ Column 'Segment' is clean
✅ Column 'Fuel_Type' is clean
✅ Column 'Transmission' is clean

Checking sheet: Sales
✅ Column 'Invoice_ID' is clean
✅ Column 'Sale_Date' is clean
✅ Column 'Dealer_ID' is clean
✅ Column 'Car_ID' is clean
✅ Column 'Currency' is clean
✅ Column 'Payment_Method' is clean

Checking sheet: Cost
✅ Column 'Car_ID' is clean

Checking sheet: Market
✅ Column 'Country' is clean
✅ Column 'Quarter' is clean


In [29]:
df_sales["Dealer_ID"].unique()

<StringArray>
['D002', 'D006', 'D003', 'D001', 'D005', 'D004']
Length: 6, dtype: str

In [30]:
df_sales["Sale_Date"] = pd.to_datetime(df_sales["Sale_Date"], errors="coerce")

print("Null dates:", df_sales["Sale_Date"].isna().sum())

Null dates: 0


In [31]:
print("Duplicate Invoice IDs:", df_sales["Invoice_ID"].duplicated().sum())

Duplicate Invoice IDs: 0


In [32]:
invalid_dealers = df_sales[~df_sales["Dealer_ID"].isin(df_dealer["Dealer_ID"])]
print("Invalid Dealer references:", len(invalid_dealers))

Invalid Dealer references: 0


In [33]:
invalid_cars = df_sales[~df_sales["Car_ID"].isin(df_car["Car_ID"])]
print("Invalid Car references:", len(invalid_cars))

Invalid Car references: 0


In [37]:
print(df_cost.columns)

Index(['Car_ID', 'Manufacturing_Cost', 'Logistics_Cost', 'Marketing_Cost',
       'Dealer_Commission_%'],
      dtype='str')


In [44]:
print(df.columns.tolist())

['Country', 'Quarter', 'Inflation_Rate_%', 'Fuel_Price', 'Interest_Rate_%', 'Exchange_Rate_to_USD']


In [45]:
df.columns = df.columns.str.strip()  # removes leading/trailing spaces
print(df.columns.tolist())  # verify cleaned names

['Country', 'Quarter', 'Inflation_Rate_%', 'Fuel_Price', 'Interest_Rate_%', 'Exchange_Rate_to_USD']


In [48]:
import pandas as pd

# ===============================
# 1. Load the Excel file
# ===============================
file_path = (r"C:\Users\hoora\OneDrive\Desktop\Project 3\Global_Automotive_Analytics_1000_Records.xlsx")  # <-- replace with your uploaded file name

# Load all sheets into a dictionary
all_sheets = pd.read_excel(file_path, sheet_name=None)  # None loads all sheets

# ===============================
# 2. Loop through all sheets and audit
# ===============================
for sheet_name, df in all_sheets.items():
    print(f"\n\n=== Auditing Sheet: {sheet_name} ===")

    # Clean column names
    df.columns = df.columns.str.strip()
    print("Columns in this sheet:", df.columns.tolist())

    # Identify numeric columns automatically
    numeric_cols = df.select_dtypes(include='number').columns.tolist()

    # -------------------------------
    # 3. Duplicates check
    # -------------------------------
    id_col = df.columns[0] if len(df.columns) > 0 else None
    if id_col:
        duplicates = df[df.duplicated(subset=[id_col], keep=False)]
        print("\nDuplicates:")
        print(duplicates if not duplicates.empty else "No duplicates found.")
    else:
        print("\nNo ID column to check duplicates.")

    # -------------------------------
    # 4. Missing values
    # -------------------------------
    missing = df.isnull().sum()
    print("\nMissing Values:")
    print(missing[missing > 0] if missing.any() else "No missing values found.")

    # -------------------------------
    # 5. Negative values check
    # -------------------------------
    negatives = df[(df[numeric_cols] < 0).any(axis=1)]
    print("\nNegative Values:")
    print(negatives if not negatives.empty else "No negative values found.")

    # -------------------------------
    # 6. Zero values check
    # -------------------------------
    zeros = df[(df[numeric_cols] == 0).any(axis=1)]
    print("\nZero Values:")
    print(zeros if not zeros.empty else "No zero values found.")

    # -------------------------------
    # 7. Data type check
    # -------------------------------
    print("\nData Types:")
    print(df.dtypes)

    # -------------------------------
    # 8. Simple outlier detection (3-sigma method)
    # -------------------------------
    for col in numeric_cols:
        mean = df[col].mean()
        std = df[col].std()
        outliers = df[(df[col] < mean - 3*std) | (df[col] > mean + 3*std)]
        if not outliers.empty:
            print(f"\nOutliers detected in {col}:")
            print(outliers[[df.columns[0], col]])

    # -------------------------------
    # 9. Preview first 5 rows
    # -------------------------------
    print("\nSample Data Preview:")
    print(df.head())



=== Auditing Sheet: Dealership_Master ===
Columns in this sheet: ['Dealer_ID', 'Dealer_Name', 'City', 'Country', 'Market_Region', 'Dealer_Type', 'Opening_Year']

Duplicates:
No duplicates found.

Missing Values:
No missing values found.

Negative Values:
No negative values found.

Zero Values:
No zero values found.

Data Types:
Dealer_ID          str
Dealer_Name        str
City               str
Country            str
Market_Region      str
Dealer_Type        str
Opening_Year     int64
dtype: object

Sample Data Preview:
  Dealer_ID             Dealer_Name          City   Country Market_Region  \
0      D001   Toyota Karachi Motors       Karachi  Pakistan      Pakistan   
1      D002    Honda Karachi Center       Karachi  Pakistan      Pakistan   
2      D003       BMW Dubai Premium         Dubai       UAE          MENA   
3      D004      Kia Abu Dhabi Auto     Abu Dhabi       UAE          MENA   
4      D005  Tesla Kuala Lumpur Hub  Kuala Lumpur  Malaysia           SEA   

  Dealer

In [51]:


# Create a new dictionary to store cleaned sheets
cleaned_sheets = {}

for sheet_name, df in all_sheets.items():
    print(f"\nProcessing sheet: {sheet_name}")

    # Clean column names
    df.columns = df.columns.str.strip()

    # -------------------------
    # Car_Master: fix Engine_CC = 0
    # -------------------------
    if sheet_name == "Car_Master" and "Engine_CC" in df.columns:
        zero_engine_rows = df["Engine_CC"] == 0
        if zero_engine_rows.any():
            df.loc[zero_engine_rows, "Engine_CC"] = "Electric/NA"
            print(f"Fixed {zero_engine_rows.sum()} Engine_CC = 0 rows.")

    # -------------------------
    # Sales_Transactions: remove or flag zero Units_Sold / Unit_Price
    # -------------------------
    if sheet_name == "Sales_Transactions":
        zero_sales = (df["Units_Sold"] == 0) | (df["Unit_Price"] == 0)
        if zero_sales.any():
            print(f"Flagging {zero_sales.sum()} zero-sales rows.")
            # Option 1: Remove these rows
            df = df[~zero_sales]
            # Option 2 (alternative): mark with a new column instead of deleting
            # df.loc[zero_sales, "Issue"] = "Zero Units/Price"

    # -------------------------
    # Market_Factors: remove duplicates
    # -------------------------
    if sheet_name == "Market_Factors":
        before = len(df)
        df = df.drop_duplicates()
        after = len(df)
        print(f"Removed {before - after} duplicate rows.")

    


Processing sheet: Dealership_Master

Processing sheet: Car_Master

Processing sheet: Sales_Transactions

Processing sheet: Cost_Profit

Processing sheet: Market_Factors
Removed 0 duplicate rows.


In [50]:
df["Engine_CC"] = df["Engine_CC"].astype(str)
zero_engine_rows = df["Engine_CC"] == "0"
df.loc[zero_engine_rows, "Engine_CC"] = "Electric/NA"

In [53]:
# Add calculated revenue column

df_sales["Calc_Revenue"] = df_sales["Units_Sold"] * df_sales["Unit_Price"] * (1 - df_sales["Discount_%"]/100)

# Flag mismatches
revenue_mismatch = df_sales[df_sales["Calc_Revenue"] != df_sales["Revenue"]]
print(f"Rows with revenue mismatch: {len(revenue_mismatch)}")

Rows with revenue mismatch: 62


In [56]:
df_sales.loc[revenue_mismatch.index, "Revenue"] = df_sales.loc[revenue_mismatch.index, "Calc_Revenue"].round(0).astype(int)

In [57]:
# Ensure Revenue column is float
df_sales["Revenue"] = df_sales["Revenue"].astype(float)

# Now fix revenue mismatches
df_sales.loc[revenue_mismatch.index, "Revenue"] = df_sales.loc[revenue_mismatch.index, "Calc_Revenue"]

print(f"✅ Fixed {len(revenue_mismatch)} revenue mismatches.")

✅ Fixed 62 revenue mismatches.


In [59]:
all_sheets = pd.read_excel(file_path, sheet_name=None)
cleaned_sheets = {name.strip().replace(" ", "_"): df for name, df in all_sheets.items()}
print(list(cleaned_sheets.keys()))

['Dealership_Master', 'Car_Master', 'Sales_Transactions', 'Cost_Profit', 'Market_Factors']


In [61]:
df_market = cleaned_sheets["Market_Factors"]
invalid_inflation = df_market[(df_market["Inflation_Rate_%"] < 0) | (df_market["Inflation_Rate_%"] > 30)]
invalid_interest = df_market[(df_market["Interest_Rate_%"] < 0) | (df_market["Interest_Rate_%"] > 20)]
print(f"Inflation issues: {len(invalid_inflation)}, Interest issues: {len(invalid_interest)}")

Inflation issues: 0, Interest issues: 0


In [62]:
for sheet_name, df in cleaned_sheets.items():
    print(f"\n=== Sheet: {sheet_name} ===")
    print(f"Rows: {len(df)}, Columns: {len(df.columns)}")
    print("Columns:", list(df.columns))
    print("Data types:\n", df.dtypes)
    print("Missing values:\n", df.isna().sum())
    print("Duplicates:", df.duplicated().sum())
    print("Sample rows:\n", df.head())


=== Sheet: Dealership_Master ===
Rows: 6, Columns: 7
Columns: ['Dealer_ID', 'Dealer_Name', 'City', 'Country', 'Market_Region', 'Dealer_Type', 'Opening_Year']
Data types:
 Dealer_ID          str
Dealer_Name        str
City               str
Country            str
Market_Region      str
Dealer_Type        str
Opening_Year     int64
dtype: object
Missing values:
 Dealer_ID        0
Dealer_Name      0
City             0
Country          0
Market_Region    0
Dealer_Type      0
Opening_Year     0
dtype: int64
Duplicates: 0
Sample rows:
   Dealer_ID             Dealer_Name          City   Country Market_Region  \
0      D001   Toyota Karachi Motors       Karachi  Pakistan      Pakistan   
1      D002    Honda Karachi Center       Karachi  Pakistan      Pakistan   
2      D003       BMW Dubai Premium         Dubai       UAE          MENA   
3      D004      Kia Abu Dhabi Auto     Abu Dhabi       UAE          MENA   
4      D005  Tesla Kuala Lumpur Hub  Kuala Lumpur  Malaysia           SEA   


In [63]:
for sheet_name, df in cleaned_sheets.items():
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        print(f"\n=== Sheet: {sheet_name} (Numeric Checks) ===")
        for col in numeric_cols:
            neg = (df[col] < 0).sum()
            zero = (df[col] == 0).sum()
            if neg > 0 or zero > 0:
                print(f"{col} => Negative: {neg}, Zero: {zero}")


=== Sheet: Dealership_Master (Numeric Checks) ===

=== Sheet: Car_Master (Numeric Checks) ===
Engine_CC => Negative: 0, Zero: 1

=== Sheet: Sales_Transactions (Numeric Checks) ===
Discount_% => Negative: 0, Zero: 180

=== Sheet: Cost_Profit (Numeric Checks) ===

=== Sheet: Market_Factors (Numeric Checks) ===


In [64]:
df_sales = cleaned_sheets["Sales_Transactions"]  # adjust name if your sheet name is different
df_sales["Calc_Revenue"] = df_sales["Units_Sold"] * df_sales["Unit_Price"] * (1 - df_sales["Discount_%"]/100)
revenue_mismatch = df_sales[df_sales["Revenue"] != df_sales["Calc_Revenue"]]
print("Revenue mismatches remaining:", len(revenue_mismatch))

Revenue mismatches remaining: 62


In [65]:
df_sales = cleaned_sheets["Sales_Transactions"]  # your sheet
df_sales["Revenue"] = df_sales["Revenue"].astype(float)

In [66]:
df_sales["Calc_Revenue"] = df_sales["Units_Sold"] * df_sales["Unit_Price"] * (1 - df_sales["Discount_%"]/100)

# Optional: round to 2 decimals
df_sales["Calc_Revenue"] = df_sales["Calc_Revenue"].round(2)

In [67]:
revenue_mismatch = df_sales[df_sales["Revenue"] != df_sales["Calc_Revenue"]]
print(f"Revenue mismatches before fix: {len(revenue_mismatch)}")

# Fix them
df_sales.loc[revenue_mismatch.index, "Revenue"] = df_sales.loc[revenue_mismatch.index, "Calc_Revenue"]

# Check again
revenue_mismatch = df_sales[df_sales["Revenue"] != df_sales["Calc_Revenue"]]
print(f"Revenue mismatches after fix: {len(revenue_mismatch)}")

Revenue mismatches before fix: 0
Revenue mismatches after fix: 0


In [68]:
# Define which columns are text in each sheet
text_cols = {
    "Dealership_Master": ["Dealer_Name", "City", "Country", "Market_Region", "Dealer_Type"],
    "Car_Master": ["Brand", "Model", "Segment", "Fuel_Type", "Transmission"],
    "Sales_Transactions": ["Dealer_ID", "Car_ID", "Currency", "Payment_Method"],
    "Cost_Profit": ["Car_ID"],
    "Market_Factors": ["Country", "Quarter"]
}

for sheet, cols in text_cols.items():
    df = cleaned_sheets[sheet]
    for col in cols:
        # Strip spaces, fix capitalization
        df[col] = df[col].astype(str).str.strip().str.title()  # title case
        # Optional: remove any unwanted characters
        df[col] = df[col].str.replace(r"[^A-Za-z0-9\s_\-]", "", regex=True)

In [73]:
audit_report = {}

for sheet_name, df in cleaned_sheets.items():
    report = {}
    report["Rows"] = len(df)
    report["Columns"] = len(df.columns)
    report["Duplicates"] = df.duplicated().sum()
    report["Missing_Values"] = df.isna().sum().to_dict()
    
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    report["Zero_Numbers"] = {col: (df[col]==0).sum() for col in numeric_cols}
    report["Negative_Numbers"] = {col: (df[col]<0).sum() for col in numeric_cols}
    
    # Revenue mismatches for Sales_Transactions
    if sheet_name == "Sales_Transactions":
        report["Revenue_Mismatches"] = len(df[df["Revenue"] != df["Calc_Revenue"]])
    
    audit_report[sheet_name] = report
for sheet, report in audit_report.items():
    print(f"\n=== {sheet} Summary ===")
    print(report)


=== Dealership_Master Summary ===
{'Rows': 6, 'Columns': 7, 'Duplicates': np.int64(0), 'Missing_Values': {'Dealer_ID': 0, 'Dealer_Name': 0, 'City': 0, 'Country': 0, 'Market_Region': 0, 'Dealer_Type': 0, 'Opening_Year': 0}, 'Zero_Numbers': {'Opening_Year': np.int64(0)}, 'Negative_Numbers': {'Opening_Year': np.int64(0)}}

=== Car_Master Summary ===
{'Rows': 6, 'Columns': 8, 'Duplicates': np.int64(0), 'Missing_Values': {'Car_ID': 0, 'Brand': 0, 'Model': 0, 'Segment': 0, 'Fuel_Type': 0, 'Transmission': 0, 'Engine_CC': 0, 'Launch_Year': 0}, 'Zero_Numbers': {'Engine_CC': np.int64(1), 'Launch_Year': np.int64(0)}, 'Negative_Numbers': {'Engine_CC': np.int64(0), 'Launch_Year': np.int64(0)}}

=== Sales_Transactions Summary ===
{'Rows': 1000, 'Columns': 11, 'Duplicates': np.int64(0), 'Missing_Values': {'Invoice_ID': 0, 'Sale_Date': 0, 'Dealer_ID': 0, 'Car_ID': 0, 'Units_Sold': 0, 'Unit_Price': 0, 'Discount_%': 0, 'Revenue': 0, 'Currency': 0, 'Payment_Method': 0, 'Calc_Revenue': 0}, 'Zero_Numbers'

In [74]:
downloads_path = r"C:\Users\hoora\Downloads\Car_Project_Cleaned.xlsx"

# Use ExcelWriter to save multiple sheets
with pd.ExcelWriter(downloads_path, engine='xlsxwriter') as writer:
    # Replace cleaned_sheets['Sheet_Name'] with your actual cleaned DataFrames
    cleaned_sheets["Dealership_Master"].to_excel(writer, sheet_name='Dealership_Master', index=False)
    cleaned_sheets["Car_Master"].to_excel(writer, sheet_name='Car_Master', index=False)
    cleaned_sheets["Sales_Transactions"].to_excel(writer, sheet_name='Sales_Transactions', index=False)
    cleaned_sheets["Cost_Profit"].to_excel(writer, sheet_name='Cost_Profit', index=False)
    cleaned_sheets["Market_Factors"].to_excel(writer, sheet_name='Market_Factors', index=False)

print(f"✅ Done! Your cleaned file is ready here:\n{downloads_path}")

✅ Done! Your cleaned file is ready here:
C:\Users\hoora\Downloads\Car_Project_Cleaned.xlsx


In [75]:
from sqlalchemy import create_engine
import pandas as pd

In [76]:
from sqlalchemy import create_engine

user = "root"
password = "Haya%40123456789!"  # URL-encoded @
host = "localhost"
port = "3306"
database = "car_project_db"

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}")
print("✅ Engine created successfully")

✅ Engine created successfully


In [78]:
# Lowercase table names to avoid warnings
sheets_to_sql = {
    "dealership_master": cleaned_sheets["Dealership_Master"],
    "car_master": cleaned_sheets["Car_Master"],
    "sales_transactions": cleaned_sheets["Sales_Transactions"],
    "cost_profit": cleaned_sheets["Cost_Profit"],
    "market_factors": cleaned_sheets["Market_Factors"]
}

for table_name, df in sheets_to_sql.items():
    df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
    print(f"✅ Table '{table_name}' transferred to SQL")

✅ Table 'dealership_master' transferred to SQL
✅ Table 'car_master' transferred to SQL
✅ Table 'sales_transactions' transferred to SQL
✅ Table 'cost_profit' transferred to SQL
✅ Table 'market_factors' transferred to SQL
